In [17]:
import numpy as np
import pandas as pd
from scipy import stats as sps
from sklearn import linear_model
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt 
import warnings
from dataclasses import dataclass
import statsmodels.api as sm
tqdm.pandas()
plt.style.use('ggplot')
pd.set_option('use_inf_as_na', True)
plt.rcParams['figure.figsize'] = (15,6)


warnings.simplefilter('ignore')


In [2]:
base_data = pd.read_pickle('../data/expN34.pkl')

In [5]:
data = base_data.query('click < view & view < view.quantile(.99)').drop('cov_device',axis=1)

In [7]:
data['ctr'] = data['click'] / data['view']

# 1 OLS as stat test

In [178]:
data_ab = data.query('part == "AB"').drop(['part'],axis=1)
data_aa = data.query('part != "AB"').drop(['part'],axis=1)

In [179]:
Y = 'ctr'
T = 'ab'
X = data.filter(regex='cov').columns.tolist()

In [236]:
def bootstrap_func(func,data,params,n_bootstraps=500):
    
    return [func(data=data.sample(frac=1,replace=True),**params) for _ in tqdm(range(n_bootstraps),position=0)]

In [243]:
def ols_ttest(data:pd.DataFrame,
              T:str,
              Y:str,
              X:list=[]):
    lm = sm.formula.ols(f'{Y}~{"+".join(X+[T])}',data=data.astype(float)).fit()
    return lm.summary2().tables[1].loc[T,'Coef.']

In [244]:
def IPV(data,
        X:list,
        T:str,
        Y:str,
        is_raw_ps=False):
    ps = linear_model.LogisticRegression(C=1e-6,penalty='none').fit(data[X], data[T]).predict_proba(data[X])[:, 1]
    weight = (data[T]-ps) / (ps*(1-ps)) # define the weights
    if is_raw_ps:
        return ps
    return (weight * data[Y]).mean() # compute the ATE

In [254]:
def doubly_robust(data, X, T, Y):
    ps = linear_model.LogisticRegression(C=1e6,penalty='none').fit(data[X], data[T]).predict_proba(data[X])[:, 1]
    mu0 = linear_model.LinearRegression().fit(data.query(f"{T}==0")[X], data.query(f"{T}==0")[Y]).predict(data[X])
    mu1 = linear_model.LinearRegression().fit(data.query(f"{T}==1")[X], data.query(f"{T}==1")[Y]).predict(data[X])

    control_metric = np.mean((1-data[T])*(data[Y] - mu0)/(1-ps) + mu0)
    treatment_metric = np.mean(data[T]*(data[Y] - mu1)/ps + mu1)

    return treatment_metric - control_metric

In [246]:
params = {
    'X':X,
    'T':T,
    'Y':Y
}

In [247]:
ols_params = {
    'X':[],
    'T':T,
    'Y':Y
}

In [261]:
base_ols_result = bootstrap_func(ols_ttest,data_aa,ols_params,n_bootstraps=100)

100%|██████████| 100/100 [00:14<00:00,  6.74it/s]


In [262]:
ipv_result = bootstrap_func(IPV,data_aa,params,n_bootstraps=100)

100%|██████████| 100/100 [01:24<00:00,  1.19it/s]


In [263]:
dr_result = bootstrap_func(doubly_robust,data_aa,params,n_bootstraps=100)

100%|██████████| 100/100 [01:09<00:00,  1.44it/s]


In [264]:
def bts_ttest(bts_data:list) -> float:
    bool_data = np.array(bts_data) <= 0
    agg_data = bool_data.mean()
    result = min(agg_data,1-agg_data) * 2
    return result

In [265]:
bts_ttest(base_ols_result)

0.98

In [266]:
bts_ttest(ipv_result)

0.8

In [267]:
bts_ttest(dr_result)

0.7